In [2]:
import numpy as np

def normalize(embed):
    m, n = embed.shape
    for i in xrange(m):
        embed[i] = embed[i] / np.linalg.norm(embed[i])
    return embed

dim = 100
with open("vectors.txt", 'r') as glove:
    words = []
    lines = glove.readlines()
    embed = np.zeros((len(lines) - 1, dim))
    body = lines[:-1]
    i = 0
    for vec in body:
        tokens = vec.split()
        words.append(tokens[0])
        embed[i] = np.asarray([float(x) for x in tokens[1:]])
        i = i + 1
embed = normalize(embed)
with open("cellvectors.txt", 'r') as cov:
    lines = cov.readlines()
    cembed = np.zeros((len(lines) - 1, dim))
    body = lines[:-1]
    i = 0
    for vec in body:
        tokens = vec.split()
        cembed[i] = np.asarray([abs(float(x)) for x in tokens[1:]])
        i = i + 1
ccembed = cembed
ncembed = normalize(cembed)
with open("timevectors.txt", 'r') as cov2:
    lines = cov2.readlines()
    tembed = np.zeros((len(lines) - 1, dim))
    body = lines[:-1]
    i = 0
    for vec in body:
        tokens = vec.split()
        tembed[i] = np.asarray([abs(float(x)) for x in tokens[1:]])
        i = i + 1
ttembed = tembed
ntembed = normalize(tembed)

In [5]:
subs = ["Conservative", "NeutralPolitics", "PoliticalDiscussion", "SandersForPresident", "The_Donald", "politics"]

for i in xrange(len(ncembed)):
    comp = subs
    dists = []
    for j in xrange(len(ncembed)):
        dists.append(np.linalg.norm(ncembed[i] - ncembed[j]))
    dists, comp = zip(*sorted(zip(dists, comp)))
    print subs[i], comp, dists

Conservative ('Conservative', 'PoliticalDiscussion', 'SandersForPresident', 'politics', 'The_Donald', 'NeutralPolitics') (0.0, 0.52643168116820505, 0.62909871513644955, 0.66819812063232942, 0.68315124790746407, 0.82581827820844056)
NeutralPolitics ('NeutralPolitics', 'PoliticalDiscussion', 'Conservative', 'SandersForPresident', 'politics', 'The_Donald') (0.0, 0.76850965306672825, 0.82581827820844056, 0.82956081564423678, 0.8817130252107509, 0.91776597506108037)
PoliticalDiscussion ('PoliticalDiscussion', 'politics', 'SandersForPresident', 'Conservative', 'The_Donald', 'NeutralPolitics') (0.0, 0.39750779591409263, 0.45585820246802711, 0.52643168116820505, 0.60072581948722392, 0.76850965306672825)
SandersForPresident ('SandersForPresident', 'PoliticalDiscussion', 'politics', 'Conservative', 'The_Donald', 'NeutralPolitics') (0.0, 0.45585820246802711, 0.47167046922819778, 0.62909871513644955, 0.64183089214965705, 0.82956081564423678)
The_Donald ('The_Donald', 'politics', 'PoliticalDiscussi

In [6]:
times = range(1, 9)

for i in xrange(len(ntembed)):
    comp = times
    dists = []
    for j in xrange(len(ntembed)):
        dists.append(np.linalg.norm(ntembed[i] - ntembed[j]))
    dists, comp = zip(*sorted(zip(dists, comp)))
    print times[i], comp, dists

1 (1, 3, 2, 6, 7, 8, 4, 5) (0.0, 0.39667295274882852, 0.40223257339612961, 0.40534936577077041, 0.45897039730543093, 0.49143925819397288, 0.49969391628012505, 0.50810580192542554)
2 (2, 3, 1, 6, 4, 8, 7, 5) (0.0, 0.34512374538947632, 0.40223257339612961, 0.40625015746483417, 0.44427047915768547, 0.45411826546847472, 0.49685125419784304, 0.52257317623401844)
3 (3, 2, 1, 7, 6, 4, 5, 8) (0.0, 0.34512374538947632, 0.39667295274882852, 0.42760928064294479, 0.43481968746188299, 0.44309901096616139, 0.50416460267502972, 0.51782377507824862)
4 (4, 3, 2, 1, 6, 5, 8, 7) (0.0, 0.44309901096616139, 0.44427047915768547, 0.49969391628012505, 0.51882687995216936, 0.55196238900599071, 0.55863542682246181, 0.56586995029454679)
5 (5, 8, 3, 1, 6, 2, 7, 4) (0.0, 0.4953856688811874, 0.50416460267502972, 0.50810580192542554, 0.51334907312206612, 0.52257317623401844, 0.54250501179069899, 0.55196238900599071)
6 (6, 1, 2, 3, 7, 8, 5, 4) (0.0, 0.40534936577077041, 0.40625015746483417, 0.43481968746188299, 0.444

# Word drift

Which words change the most from subreddit to subreddit?

In [7]:
sub_embed = {}
for k in xrange(len(subs)):
    m, n = embed.shape
    sub_specific_embed = np.zeros((m, n))
    for i in xrange(m):
        sub_specific_embed[i] = np.multiply(embed[i], ccembed[k])
    sub_embed[subs[k]] = normalize(sub_specific_embed)

In [26]:
def most_drift(sub1, sub2, w):
    wp = w
    m, n = embed.shape
    sub1_embed = sub_embed[sub1]
    sub2_embed = sub_embed[sub2]
    drifts = [np.linalg.norm(np.asarray([np.linalg.norm(sub1_embed[i] - sub1_embed[j]) for j in xrange(1000)]) - np.asarray([np.linalg.norm(sub2_embed[i] - sub2_embed[j]) for j in xrange(1000)])) for i in xrange(1000)]
    drifts, wp = zip(*sorted(zip(drifts, wp)))
    return drifts, wp
            

In [27]:
a, b = most_drift("politics", "The_Donald", words)

In [73]:
def sub_task(embed, sub_embed, words, subs, check):
    # find pairs of words which are pushed together in a particular sub
    sub_squeeze = {}
    for k in xrange(len(subs)):
        print k
        sembed = sub_embed[subs[k]]
        sort_pairs, sort_ranges = [], []
        for i in xrange(len(check)):
            for j in xrange(i):
                sort_pairs.append((words[check[i]], words[check[j]]))
                shift = np.linalg.norm(sembed[check[i]] - sembed[check[j]]) - np.linalg.norm(embed[check[i]] - embed[check[j]])
                sort_ranges.append(shift)
        sort_ranges, sort_pairs = zip(*sorted(zip(sort_ranges, sort_pairs)))
        sub_squeeze[subs[k]] = (sort_ranges, sort_pairs)
    return sub_squeeze

In [80]:
a = sub_task(embed, sub_embed, words, subs, common_indices)

0
1
2
3
4
5


In [78]:
import pickle
common_words = pickle.load(open("common_words.p", 'rb'))

In [79]:
word_dict = {words[i] : i for i in xrange(len(words))}
common_indices = [word_dict[word] for word in common_words]

In [81]:
trump_push = a["The_Donald"][1][::-1]
sanders_push = a["SandersForPresident"][1][::-1]

In [83]:
trump_push[:100]

(('blatantly', 'yup'),
 ('target', 'although'),
 ('p', 'previously'),
 ('suffer', 'proposed'),
 ('lock', 'negative'),
 ('included', 'offer'),
 ('anyways', 'building'),
 ('choices', 'peace'),
 ('endorsements', 'via'),
 ('reduce', 'thus'),
 ('kicked', 'among'),
 ('limit', 'pathetic'),
 ('x', 'yep'),
 ('filed', 'letter'),
 ('english', 'project'),
 ('denying', 'backing'),
 ('increasing', 'requires'),
 ('offices', 'illinois'),
 ('letter', 'michigan'),
 ('requires', 'played'),
 ('counted', 'progressive'),
 ('thedonald', 'controversial'),
 ('strike', 'fake'),
 ('brown', 'affect'),
 ('named', 'keeping'),
 ('rthedonald', 'cultural'),
 ('responsibility', 'likes'),
 ('strange', 'pathetic'),
 ('backed', 'debt'),
 ('math', 'join'),
 ('choosing', 'leaks'),
 ('pays', 'training'),
 ('538', 'progress'),
 ('sane', 'activity'),
 ('rational', 'nations'),
 ('historical', 'quit'),
 ('canvassing', 'kinda'),
 ('hole', 'statements'),
 ('arrested', 'georgia'),
 ('georgia', 'society'),
 ('statistics', 'nations')

In [68]:
common_words[:500]

('the',
 'to',
 'a',
 'and',
 'of',
 'is',
 'that',
 'in',
 'i',
 'it',
 'for',
 'you',
 'this',
 'be',
 'if',
 'on',
 'are',
 'they',
 'he',
 'not',
 'have',
 'but',
 'with',
 'was',
 'as',
 'trump',
 'people',
 'its',
 'just',
 'we',
 'like',
 'or',
 'about',
 'would',
 'so',
 'all',
 'at',
 'has',
 'what',
 'will',
 'his',
 'from',
 'by',
 'she',
 'dont',
 'who',
 'her',
 'more',
 'an',
 'their',
 'do',
 'can',
 'out',
 'no',
 'because',
 'up',
 'think',
 'how',
 'get',
 'there',
 'one',
 'them',
 'when',
 'my',
 'hillary',
 'than',
 'even',
 'were',
 'clinton',
 'im',
 'him',
 'your',
 'been',
 'only',
 'me',
 'some',
 'vote',
 'know',
 'going',
 'bernie',
 'now',
 'time',
 'unfit',
 'thats',
 'then',
 'being',
 'had',
 'really',
 'any',
 'make',
 'us',
 'want',
 'other',
 'should',
 'right',
 'also',
 'hes',
 'much',
 'why',
 'see',
 'which',
 'could',
 'way',
 'our',
 'say',
 'most',
 'over',
 'said',
 'doesnt',
 'well',
 'good',
 'party',
 'those',
 'sanders',
 'these',
 'go',
 

In [69]:
common_indices[:10]

[12287, 20558, 33066, 37983, 18004, 14747, 32908, 32766, 14167, 32692]

In [70]:
word_dict['the']

12287